# REST API
In this notebook we will discuss how to use REST API to pull data. We assume the app id is authenticate with secret token.

In [10]:
# install all the azure identity modules
%pip install azure.identity
# install all the azure keyvault modules
%pip install azure-keyvault-secrets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from azure.identity import AzureCliCredential, ChainedTokenCredential, DefaultAzureCredential, ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
from azure.identity import ClientSecretCredential
import requests;

class KeyVault():
    def __init__(self, configuration : dict):
        key_vault = configuration.get('keyvault');
        credential = ChainedTokenCredential(ManagedIdentityCredential(), 
                                            AzureCliCredential(), 
                                            DefaultAzureCredential());
        kv_url = "https://{key_vault}.vault.azure.net";
        kv_url = kv_url.format(key_vault = key_vault);
        self.client = SecretClient(vault_url=kv_url, credential=credential)

    # write a database to database, mode = append or replace
    def get(self, secret_name : str) -> str:
        secret_value = self.client.get_secret(secret_name);
        return secret_value.value;


class RestAPI():
    def __init__(self, configuration : dict):
        app_id = configuration.get('app_id');
        app_secret = configuration.get('app_secret');
        tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47';
        credential = ClientSecretCredential(tenant_id, app_id, app_secret);
        self.token = credential.get_token("280b00f4-2c5a-44d8-bab9-6acbbaaa7c78/.default").token;
    
    def call_data_api(self, uri : str) -> dict:
        result = [];
        try:
            headers = {
                "Authorization": "Bearer " + self.token,
                "Content-Type": "application/json"  # Adjust content type as per your API
            }
            response = requests.get(uri, headers=headers);
            if response.status_code == 200:
                result = response.json();
            else:
                print("Error:", response.status_code)
        except requests.exceptions.RequestException as e:
            print("Error:", e);
        return result;

key_vault = KeyVault({'keyvault' : 'etlframeworkkeyvaults'});
app_id = key_vault.get('sec-CoiDpxeETLClientId');
app_secret = key_vault.get('sec-CoiDpxeETLClientKey');
api_client = RestAPI({'app_id' : app_id, 'app_secret': app_secret});
result = api_client.call_data_api("https://api.infra.global.azure.com/pantheon/api/v2/entities/AGI_Projects")
projects = result['result'];
print(projects)



Error: 400


TypeError: list indices must be integers or slices, not str